In [1]:
import pandas as pd

In [2]:
deck_file = '../../data/OTJ/deck.feather'
card_file = '../../data/OTJ/card.feather'

In [3]:

df_d = pd.read_feather(deck_file)
df_d.shape

(1167709, 399)

In [4]:
df_d[:100].columns.to_list()[:15]

['expansion',
 'event_type',
 'build_index',
 'match_number',
 'game_number',
 'rank',
 'opp_rank',
 'main_colors',
 'splash_colors',
 'on_play',
 'num_mulligans',
 'opp_num_mulligans',
 'opp_colors',
 'num_turns',
 'won']

In [5]:
# sample rows

n_rows = 150_000  # ~1.1M rows originally

print(df_d.shape)
df_d = df_d.sample(n_rows, random_state=42)
print(df_d.shape)

(1167709, 399)
(150000, 399)


In [6]:
card_st_idx = df_d.columns.to_list().index('Abraded Bluffs')
card_cols = df_d.columns[card_st_idx:].to_list()
print(len(card_cols))

n_cards = 100
df_col = df_d[card_cols][:1].sample(n_cards, axis=1, random_state=42)
card_cols = df_col.columns
card_cols


381


Index(['Razzle-Dazzler', 'Magda, the Hoardmaster', 'Bucolic Ranch',
       'Clear Shot', 'Collector's Cage', 'Bridled Bighorn',
       'Iron-Fist Pulverizer', 'Vadmir, New Blood', 'Mountain',
       'Eriette's Lullaby', 'Spring Splasher', 'Obeka, Splitter of Seconds',
       'Rambling Possum', 'Intimidation Campaign', 'Daring Thunder-Thief',
       'Malcolm, the Eyes', 'Detention Sphere', 'The Key to the Vault',
       'Plains', 'Mindbreak Trap', 'Unfortunate Accident',
       'Geralf, the Fleshwright', 'Arid Archway', 'Return the Favor',
       'Trash the Town', 'Primal Might', 'Abraded Bluffs', 'Fell the Mighty',
       'Tyrant's Scorn', 'Bedevil', 'Boom Box', 'Annie Joins Up',
       'Demonic Ruckus', 'Scalestorm Summoner', 'Torpor Orb',
       'Djinn of Fool's Fall', 'Final Showdown', 'Outlaw Stitcher',
       'Bristlepack Sentry', 'Sterling Hound', 'Plan the Heist', 'Fling',
       'Skewer the Critics', 'Villainous Wealth', 'Bandit's Haul',
       'Boneyard Desecrator', 'Deepmuck 

In [7]:
df_d[card_cols] = df_d[card_cols] > 0

In [8]:
idx_cols = ['draft_idx', 'won']
cols = idx_cols + card_cols.to_list()
df_d = df_d[cols]
df_d.shape


(150000, 102)

In [9]:
df_d_melt = df_d[cols].melt(id_vars=idx_cols, var_name='card', value_name='in_deck')
df_d_melt.to_feather('../../data/OTJ/deck_melt.feather')
df_d_melt.shape

(15000000, 4)

In [10]:
df_d_melt.sample(5)

,draft_idx,won,card,in_deck
2211732,30564,False,Daring Thunder-Thief,False
10441594,92866,False,At Knifepoint,False
7277974,160274,True,Collective Defiance,False
5414483,130260,True,Final Showdown,False
12957016,127348,True,Smuggler's Surprise,False


In [11]:
df_d_melt['in_deck'].mean()

0.07281046666666667

In [16]:
df_c = pd.read_feather(card_file)
df_c.shape


(371, 17)

In [17]:
df = pd.merge(df_d_melt, df_c, left_on='card', right_on='name', how='left')
df.shape

(15000000, 21)

In [18]:
na_mask = df['name'].isna()
print(na_mask.sum())
if na_mask.sum() > 0:
    df[na_mask].sample(10, random_state=42)

300000


In [19]:
df.head()

,draft_idx,won,card,in_deck,name,setCode,releaseDate,number,layout,availability,...,toughness,colorIdentity,colors,types,subtypes,supertypes,manaCost,manaValue,edhrecRank,edhrecSaltiness
0,96414,True,Razzle-Dazzler,False,Razzle-Dazzler,OTJ,2024-04-19,63,normal,"arena, mtgo, paper",...,2,U,U,Creature,"Human, Wizard",,{1}{U},2.0,17822.0,NaN
1,170600,True,Razzle-Dazzler,False,Razzle-Dazzler,OTJ,2024-04-19,63,normal,"arena, mtgo, paper",...,2,U,U,Creature,"Human, Wizard",,{1}{U},2.0,17822.0,NaN
2,157994,True,Razzle-Dazzler,False,Razzle-Dazzler,OTJ,2024-04-19,63,normal,"arena, mtgo, paper",...,2,U,U,Creature,"Human, Wizard",,{1}{U},2.0,17822.0,NaN
3,35865,True,Razzle-Dazzler,False,Razzle-Dazzler,OTJ,2024-04-19,63,normal,"arena, mtgo, paper",...,2,U,U,Creature,"Human, Wizard",,{1}{U},2.0,17822.0,NaN
4,1123,True,Razzle-Dazzler,False,Razzle-Dazzler,OTJ,2024-04-19,63,normal,"arena, mtgo, paper",...,2,U,U,Creature,"Human, Wizard",,{1}{U},2.0,17822.0,NaN


In [21]:
df.to_feather('../../data/OTJ/deck_melt_card.feather')